In [11]:
import json
import numpy as np
import cv2

In [12]:
with open('saved_frame_data/T_1_everything.json') as data_file:    
    all_data = json.load(data_file)

In [13]:
with open('saved_frame_data/T_1_labels.json') as data_file:    
    all_labels = json.load(data_file)

In [14]:
def get_optical_flow(path1, path2, c):
    if path2 == "end":
        return "nah"
    frame1 = cv2.imread(path1)
    frame2 = cv2.imread(path2)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    nxt = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,nxt, None, 0.5, 3, 15, 3, 7, 1.2, 0)
    return flow

In [15]:
def get_optical_flow_vector(flow, box, X_SIZE, Y_SIZE):
    if flow == "nah":
        return "nah"
    if flow != []:
        box = [int(box[0]*Y_SIZE),int(box[1]*X_SIZE), int(box[2]*Y_SIZE), int(box[3]*X_SIZE)]
        xflow = []
        yflow = []
        for i in range(box[1],box[3]):
            for j in range(box[0],box[2]):
                xflow.append(flow[i][j][0])
                yflow.append(flow[i][j][1])
        #boxflow = [xflow,yflow]
        boxflow = [yflow,xflow]
        #print boxflow
        x = np.mean(boxflow[0])
        y = np.mean(boxflow[1])
        return [x,y]
    else:
        return None

In [16]:
def get_optical_flow_vector2(flow, box, X_SIZE, Y_SIZE):
    if flow == "nah":
        return "nah"
    l_count_x = []; l_count_y = []; r_count_x = []; r_count_y = []
    if flow != []:
        box = [int(box[0]*Y_SIZE),int(box[1]*X_SIZE), int(box[2]*Y_SIZE), int(box[3]*X_SIZE)]
        #print(box)
        for i in range(box[1],box[3]):
            for j in range(box[0],box[2]):
                if bucket_vectors(flow[i][j]) is "left":
                    l_count_x.append(flow[i][j][0])
                    l_count_y.append(flow[i][j][1])
                else:
                    r_count_x.append(flow[i][j][0])
                    r_count_y.append(flow[i][j][1])
        if len(l_count_x) >= len(r_count_x):
            #boxflow = [l_count_x,l_count_y]
            boxflow = [l_count_y,l_count_x]
        else:
            #boxflow = [r_count_x,r_count_y]
            boxflow = [r_count_y,r_count_x]
        #print boxflow
        x = np.mean(boxflow[0])
        y = np.mean(boxflow[1])
        return [x,y]
    else:
        return None

In [17]:
def bucket_vectors(vect):
    if vect == "nah":
        return "nah"
    v1 = np.array([vect[0]])
    v2 = np.array([vect[1]])
    mag, ang = cv2.cartToPolar(v1, v2, angleInDegrees = 1)
    if ang > 90 and ang < 270:
        return "left"
    else:
        return "right"

In [18]:
path = 'video_frames/frame'
base_img = cv2.imread(path+"0.jpg")
X_SIZE, Y_SIZE, channels = base_img.shape
last_file = 10
new_data = []
print(len(all_data))
for i in range(len(all_data)):
    data_elem = all_data[i]
    box = data_elem['box']
    f = data_elem['file']
    current_frame = int(f[f.find('/frame')+len('/frame'):f.find('.jpg')])
    if (current_frame + 6 >= last_file):
        flow = "nah"
    else:
        flow = get_optical_flow(path + str(current_frame) +'.jpg', path+str(current_frame+6)+'.jpg',0)
    flow1 = get_optical_flow_vector(flow, box, X_SIZE, Y_SIZE)
    flow2 = get_optical_flow_vector2(flow, box, X_SIZE, Y_SIZE)
    flow1_dir = bucket_vectors(flow1)
    flow2_dir = bucket_vectors(flow2)
    if (current_frame + 6 >= last_file):
        data_elem['flow'] = "nah"
        data_elem['flow2'] = "nah"
    else:
        data_elem['flow'] = [float(flow1[0]), float(flow1[1])]
        data_elem['flow2'] = [float(flow2[0]), float(flow2[1])]
    data_elem['direction'] = flow1_dir
    data_elem['direction2'] = flow2_dir
    new_data.append(data_elem)

42435


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  """


In [19]:
len(new_data)

42435

In [20]:
with open('T_1_2_everything.json', 'w') as fp:
    json.dump(new_data, fp)